In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/humor-assessing/train_task1.csv
/kaggle/input/humor-assessing/train_task2.csv
/kaggle/input/humor-assessing/dev_task1.csv
/kaggle/input/humor-assessing/dev_task2.csv


# Import required libraries

In [18]:
import re
import string
import nltk
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
!pip install bert-for-tf2
import bert


# Read data

In [19]:
data = pd.read_csv('/kaggle/input/humor-assessing/train_task1.csv')
data

,id,original,edit,grades,meanGrade
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0
...,...,...,...,...,...
9647,10899,State officials blast ' unprecedented ' DHS <m...,idea,0,0.0
9648,1781,Protesters Rally for <Refugees/> Detained at J...,stewardesses,20000,0.4
9649,5628,Cruise line Carnival Corp. joins the fight aga...,raisin,21000,0.6
9650,14483,Columbia police hunt woman seen with <gun/> ne...,cake,32200,1.4


# Replace the original words with edit words in the headline

In [20]:
expr = re.compile("<(.*?)/>")
data['edited_headline'] = ""
for i in range(data.shape[0]):
    data.edited_headline[i] = re.sub(expr, data.edit[i], data.original[i])
data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,original,edit,grades,meanGrade,edited_headline
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2,France is ‘ hunting down its citizens who join...
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6,"Pentagon claims 2,000 % increase in Russian tr..."
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0,Iceland PM Calls Snap Vote as Pedophile Furor ...
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4,"In an apparent first , Iran and Israel slap ea..."
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0,Trump was told weeks ago that Flynn misled sch...
...,...,...,...,...,...,...
9647,10899,State officials blast ' unprecedented ' DHS <m...,idea,0,0.0,State officials blast ' unprecedented ' DHS id...
9648,1781,Protesters Rally for <Refugees/> Detained at J...,stewardesses,20000,0.4,Protesters Rally for stewardesses Detained at ...
9649,5628,Cruise line Carnival Corp. joins the fight aga...,raisin,21000,0.6,Cruise line Carnival Corp. joins the fight aga...
9650,14483,Columbia police hunt woman seen with <gun/> ne...,cake,32200,1.4,Columbia police hunt woman seen with cake near...


# Data cleaning
## Remove punctuation

In [21]:
data['edited_headline'] = data['edited_headline'].str.lower()
table = str.maketrans('', '', string.punctuation)
data['edited_headline'] = [data['edited_headline'][row].translate(table) for row in range(len(data['edited_headline']))]
data

,id,original,edit,grades,meanGrade,edited_headline
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2,france is ‘ hunting down its citizens who join...
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6,pentagon claims 2000 increase in russian trol...
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0,iceland pm calls snap vote as pedophile furor ...
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4,in an apparent first iran and israel slap eac...
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0,trump was told weeks ago that flynn misled sch...
...,...,...,...,...,...,...
9647,10899,State officials blast ' unprecedented ' DHS <m...,idea,0,0.0,state officials blast unprecedented dhs idea...
9648,1781,Protesters Rally for <Refugees/> Detained at J...,stewardesses,20000,0.4,protesters rally for stewardesses detained at ...
9649,5628,Cruise line Carnival Corp. joins the fight aga...,raisin,21000,0.6,cruise line carnival corp joins the fight agai...
9650,14483,Columbia police hunt woman seen with <gun/> ne...,cake,32200,1.4,columbia police hunt woman seen with cake near...


## Remove stopwords

In [22]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['edited_headline'] = data['edited_headline'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,original,edit,grades,meanGrade,edited_headline
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2,france ‘ hunting citizens joined twins ’ witho...
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6,pentagon claims 2000 increase russian trolls b...
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0,iceland pm calls snap vote pedophile furor cra...
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4,apparent first iran israel slap militarily
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0,trump told weeks ago flynn misled school presi...
...,...,...,...,...,...,...
9647,10899,State officials blast ' unprecedented ' DHS <m...,idea,0,0.0,state officials blast unprecedented dhs idea s...
9648,1781,Protesters Rally for <Refugees/> Detained at J...,stewardesses,20000,0.4,protesters rally stewardesses detained jfk air...
9649,5628,Cruise line Carnival Corp. joins the fight aga...,raisin,21000,0.6,cruise line carnival corp joins fight bermuda ...
9650,14483,Columbia police hunt woman seen with <gun/> ne...,cake,32200,1.4,columbia police hunt woman seen cake near univ...


# Using BERT for word-embeddings

In [23]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [24]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [25]:
MAX_LEN = 64
train_input = bert_encode(data.edited_headline.values, tokenizer, max_len=MAX_LEN)
train_input

(array([[  101,  2605,  1520, ...,     0,     0,     0],
        [  101, 20864,  4447, ...,     0,     0,     0],
        [  101, 10399,  7610, ...,     0,     0,     0],
        ...,
        [  101,  8592,  2240, ...,     0,     0,     0],
        [  101,  3996,  2610, ...,     0,     0,     0],
        [  101,  2160, 29098, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [26]:
test = pd.read_csv('/kaggle/input/humor-assessing/dev_task1.csv')
test

,id,original,edit
0,1723,Thousands of gay and bisexual <men/> convicted...,swans
1,12736,Special <prosecutor/> appointed to Trump Russia,chef
2,12274,Spanish police detain man and search Ripoll ad...,squad
3,8823,N.Y. Times <reprimands/> reporter for sharing ...,applauds
4,5087,Vladimir Putin Releases Video Simulation Of Ru...,balloon
...,...,...,...
2414,1202,Supreme <Court/> Once Again Strikes Down Racia...,leaders
2415,14764,Trump Mocks Schumer ’s Tears ; Vows to ‘ Make ...,Insane
2416,12595,US government memo on the <danger/> of leaking...,amusement
2417,70,Newt Gingrich : Join Me in Supporting Judge Ro...,Molest


In [27]:
test['edited_headline'] = ""
for i in range(test.shape[0]):
    test.edited_headline[i] = re.sub(expr, test.edit[i], test.original[i])
test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,original,edit,edited_headline
0,1723,Thousands of gay and bisexual <men/> convicted...,swans,Thousands of gay and bisexual swans convicted ...
1,12736,Special <prosecutor/> appointed to Trump Russia,chef,Special chef appointed to Trump Russia
2,12274,Spanish police detain man and search Ripoll ad...,squad,Spanish police detain man and search Ripoll ad...
3,8823,N.Y. Times <reprimands/> reporter for sharing ...,applauds,N.Y. Times applauds reporter for sharing ' unf...
4,5087,Vladimir Putin Releases Video Simulation Of Ru...,balloon,Vladimir Putin Releases Video Simulation Of Ru...
...,...,...,...,...
2414,1202,Supreme <Court/> Once Again Strikes Down Racia...,leaders,Supreme leaders Once Again Strikes Down Racial...
2415,14764,Trump Mocks Schumer ’s Tears ; Vows to ‘ Make ...,Insane,Trump Mocks Schumer ’s Tears ; Vows to ‘ Make ...
2416,12595,US government memo on the <danger/> of leaking...,amusement,US government memo on the amusement of leaking...
2417,70,Newt Gingrich : Join Me in Supporting Judge Ro...,Molest,Newt Gingrich : Join Me in Supporting Judge Ro...


In [28]:
test['edited_headline'] = test['edited_headline'].str.lower()
table = str.maketrans('', '', string.punctuation)
test['edited_headline'] = [test['edited_headline'][row].translate(table) for row in range(len(test['edited_headline']))]
test

,id,original,edit,edited_headline
0,1723,Thousands of gay and bisexual <men/> convicted...,swans,thousands of gay and bisexual swans convicted ...
1,12736,Special <prosecutor/> appointed to Trump Russia,chef,special chef appointed to trump russia
2,12274,Spanish police detain man and search Ripoll ad...,squad,spanish police detain man and search ripoll ad...
3,8823,N.Y. Times <reprimands/> reporter for sharing ...,applauds,ny times applauds reporter for sharing unfoun...
4,5087,Vladimir Putin Releases Video Simulation Of Ru...,balloon,vladimir putin releases video simulation of ru...
...,...,...,...,...
2414,1202,Supreme <Court/> Once Again Strikes Down Racia...,leaders,supreme leaders once again strikes down racial...
2415,14764,Trump Mocks Schumer ’s Tears ; Vows to ‘ Make ...,Insane,trump mocks schumer ’s tears vows to ‘ make a...
2416,12595,US government memo on the <danger/> of leaking...,amusement,us government memo on the amusement of leaking...
2417,70,Newt Gingrich : Join Me in Supporting Judge Ro...,Molest,newt gingrich join me in supporting judge roy...


In [29]:
test['edited_headline'] = test['edited_headline'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test

,id,original,edit,edited_headline
0,1723,Thousands of gay and bisexual <men/> convicted...,swans,thousands gay bisexual swans convicted longabo...
1,12736,Special <prosecutor/> appointed to Trump Russia,chef,special chef appointed trump russia
2,12274,Spanish police detain man and search Ripoll ad...,squad,spanish police detain man search ripoll addres...
3,8823,N.Y. Times <reprimands/> reporter for sharing ...,applauds,ny times applauds reporter sharing unfounded r...
4,5087,Vladimir Putin Releases Video Simulation Of Ru...,balloon,vladimir putin releases video simulation russi...
...,...,...,...,...
2414,1202,Supreme <Court/> Once Again Strikes Down Racia...,leaders,supreme leaders strikes racial gerrymandering ...
2415,14764,Trump Mocks Schumer ’s Tears ; Vows to ‘ Make ...,Insane,trump mocks schumer ’s tears vows ‘ make ameri...
2416,12595,US government memo on the <danger/> of leaking...,amusement,us government memo amusement leaking media leaked
2417,70,Newt Gingrich : Join Me in Supporting Judge Ro...,Molest,newt gingrich join supporting judge roy moore ...
